### Importing Important Libraries

In [3]:
pip install pg8000

     ---------------------------------------- 0.0/54.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/54.5 kB ? eta -:--:--
     ------- -------------------------------- 10.2/54.5 kB ? eta -:--:--
     ------- -------------------------------- 10.2/54.5 kB ? eta -:--:--
     ------- -------------------------------- 10.2/54.5 kB ? eta -:--:--
     ------- -------------------------------- 10.2/54.5 kB ? eta -:--:--
     ------- -------------------------------- 10.2/54.5 kB ? eta -:--:--
     ---------------------------- --------- 41.0/54.5 kB 130.7 kB/s eta 0:00:01
     ----------------------------------- -- 51.2/54.5 kB 145.8 kB/s eta 0:00:01
     ----------------------------------- -- 51.2/54.5 kB 145.8 kB/s eta 0:00:01
     ----------------------------------- -- 51.2/54.5 kB 145.8 kB/s eta 0:00:01
     --------------------------------------- 54.5/54.5 kB 88.5 kB/s eta 0:00:00
     ---------------------------------------- 0.0/105.0 kB ? eta -:--:--
     --- ---------


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: c:\Users\Admin\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Add the scripts folder to the system path so we can import time_series.py
import sys
import os
from scipy.stats import zscore
# for a folder that contain scripts for modularity 
sys.path.append(os.path.abspath('../scripts'))
sys.path.append(os.path.abspath('../src')) 
from preprocessing import NoiseHandling
from data_loader import FileLoader
from path import FilePathManager
from connection import Db_Connection
from eda import EDA
from user_overview_analysis import UserOverviewAnalysis
from user_engagement_analysis import user_engagement_analysis
from user_satisfaction_analysis import UserSatisfactionAnalysis
from plot import Plot

#### Here I connect the postgress database then I read data from the table `xdr_data` then transform it to dataframe store it on a variable df